In [1]:
import pandas as pd
# read data
length_acce = pd.read_table('/home/ouconstand/Learning/graduation/virus_host/construct_graph/Virion_accession_length.txt', sep='\t')
acce_id = pd.read_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/final_acce_virusID.csv')

length_acce.columns = ['Accession','Length']
# move duplication
length_acce.drop_duplicates(['Accession','Length'],inplace=True)

In [2]:
old_data = set(length_acce['Accession'])
new_data = set(acce_id['Accession'])
print(len(old_data))
print(len(new_data))

1023868
1025047


In [3]:
different_data = list((old_data | new_data)-old_data)
len(different_data)

1290

In [23]:
# download discrepant data

import pandas as pd
import requests
import re
from bs4 import BeautifulSoup

# the function is designed to get fasta format information
api_key = '59cbd6ff6b3ce5d506f0eeb69e4e09362409'
def get_url_info(segment):

    # Using requests function get the id from term
    term_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi?db=nucleotide&api_key={}&term='.format(api_key) + ",".join(segment)
    term_info = BeautifulSoup(requests.get(term_url, timeout=(10,30)).text, 'html.parser')

    # get id
    pattern = re.compile(r'<id>(.*)</id>')
    accession_id = re.findall(pattern, str(term_info))
    
    # get sequencing information from id
    fasta_url = 'https://eutils.ncbi.nlm.nih.gov/entrez/eutils/efetch.fcgi?db=nucleotide&rettype=fasta&id={}&api_key={}'.format(','.join(accession_id), api_key)
    fasta_info = requests.get(fasta_url, timeout=(10,30)).text
    return fasta_info

# this function is designed to write files
def write_file(fasta_info,name_num):
    with open("/home/ouconstand/data/Virus_Host/sequence_{}.fasta".format(name_num), 'w') as f:
        f.write(fasta_info)

In [4]:
whole_data = pd.read_table('/home/ouconstand/Learning/graduation/virus_host/construct_graph/merge_added_raw_seq_len.txt', sep='\t')
whole_data.columns = ['Accession', 'Length']
whole_data.shape

(1026559, 2)

In [5]:
fuck_add = list((set(whole_data['Accession']) | new_data) - set(whole_data['Accession']))
len(fuck_add)

0

In [26]:
for i in range(2):
    write_file(get_url_info(fuck_add[i*20:(i+1)*20]),i)

In [12]:
acce_id_len = pd.merge(acce_id, whole_data, on='Accession', how='inner')
print(acce_id_len.shape)
acce_id_len.drop_duplicates(subset=['Accession','VirusID','Length'],inplace=True)
print(acce_id_len.shape)
acce_id_len.dropna(inplace=True)

(1026459, 3)
(1025056, 3)


In [7]:
len(set(acce_id_len['VirusID']))

953

In [10]:
acce_id_len.to_csv('/home/ouconstand/Learning/graduation/virus_host/construct_graph/acce_id_len.csv', index=None)

In [20]:
max_len_ID = acce_id_len.groupby("VirusID").Length.agg(["max"])
max_len_ID = max_len_ID.reset_index()
max_len_ID.columns = ['VirusID', 'Length']
max_len_ID

,VirusID,Length
0,11137,28754
1,28295,28124
2,229103,88
3,277944,27832
4,290028,29983
...,...,...
948,2856927,387
949,2856928,387
950,2856929,387
951,2856930,387


In [24]:
only_maxlen_id_acce = pd.merge(max_len_ID, acce_id_len, on=['VirusID','Length'], how='inner')
only_maxlen_id_acce
only_maxlen_id_acce.drop_duplicates(subset=['VirusID', 'Length'], inplace=True)

In [28]:
only_maxlen_id_acce.to_csv('maxlen_acce_id.csv', index=None)

In [29]:
only_maxlen_id_acce[only_maxlen_id_acce['Length'] > 20000]

,VirusID,Length,Accession
0,11137,28754,KT253271.1
1,28295,28124,KX981440.1
7,277944,27832,MG428704.1
8,290028,29983,KF686343.1
37,572290,26396,NC_011549.1
...,...,...,...
1211,2814912,26289,MW345815.1
1212,2814913,26208,MW345816.1
1213,2814914,26240,MW349841.1
1216,2836123,28878,MZ190137.1


In [ ]:
! cat maxlen_acce_id.csv | awk 'NR!=1 {print}' | awk -F ',' '$2<20000 {print $3}' > less_than_20000_acce.txt